In [2]:
import os
import random

import cv2
import json
import matplotlib.pyplot as plt
import numpy as np
import yaml

import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
import detectron2.utils.comm as comm
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

from detectron2.data import build_detection_train_loader
from detectron2.data import build_detection_test_loader

geladas_root = os.path.dirname(os.path.dirname(os.getcwd()))
local_paths_file = os.path.join(geladas_root, 'local-paths.json')
with open(local_paths_file, "r") as json_file:
    local_paths = json.load(json_file)

import sys
sys.path.append(
    os.path.join(local_paths["general_dection_path"],
                 "detectron2_classes"
                )
)
from DetectionDatasetMapper import DetectionDatasetMapper
from DetectionTrainer import DetectionTrainer

In [2]:
# Was designed with 0.2.1
detectron2.__version__

'0.2.1'

In [3]:
base_folder = local_paths["base_gelada"]

In [4]:
images_folder = os.path.join(base_folder, "annotation", "annotated_images")
train_json = os.path.join(base_folder, "annotation", "train.json") 
val_json = os.path.join(base_folder, "annotation", "val.json")

register_coco_instances("train", {}, train_json, images_folder)
register_coco_instances("val", {}, val_json, images_folder)

In [5]:
train_metadata = MetadataCatalog.get("train")
train_dicts = DatasetCatalog.get("train")
val_dicts = DatasetCatalog.get("val")

In [6]:
num_train_images = len(train_dicts)
num_val_images = len(val_dicts)
print(f"{num_train_images} training images \n"
      + f"{num_val_images} validation images")

train_metadata = MetadataCatalog.get("train")
print(f"Annotated classes: {train_metadata.get('thing_classes')}")

69 training images 
13 validation images


['gelada', 'human']

In [7]:
# To identify model
name = "LRscheduler-cropped-color-aug-test-new-dataloader"

cfg = get_cfg()
cfg.merge_from_file(
    os.path.join(local_paths["detectron_path"],
                 "configs", "COCO-Detection",
                 "faster_rcnn_R_50_FPN_3x.yaml")
)

cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("val",)
# CHANGE BASED ON THE NUMBER OF CORES ON COMPUTER
cfg.DATALOADER.NUM_WORKERS = 6
cfg.DATALOADER.ASPECT_RATIO_GROUPING = False
# Use pre-trained weights for transfer learning
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
)
# Change batch size based on amount of memory in GPU
# (Make the number smaller if getting GPU memory errors)
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.0019
# Maximum number of times to cycle through training data while training
max_epochs = 10
iter_per_epoch = num_train_images // cfg.SOLVER.IMS_PER_BATCH
cfg.SOLVER.MAX_ITER = max_epochs * iter_per_epoch
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupValLossLR"
# How often to save a checkpoint while training
# 0 to save best model based on minimum val loss
cfg.SOLVER.CHECKPOINT_PERIOD = 0 

cfg.SOLVER.WARMUP_ITERS = 5 
# This number should be bigger than the number
# of expected objects in each frame
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (256)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(train_metadata.get("thing_classes"))
# How often to check model performance on the validation set
# if iter_per_epoch then every epoch
cfg.TEST.EVAL_PERIOD = iter_per_epoch 
# Should be larger than max number of expected objects in image
cfg.TEST.DETECTIONS_PER_IMAGE = 200

cfg.INPUT.MIN_SIZE_TRAIN = (0)
cfg.INPUT.MIN_SIZE_TEST = (0)
cfg.INPUT.MAX_SIZE_TRAIN = (4100)
cfg.INPUT.MAX_SIZE_TEST = (4100)

# Set the data augementation regime
cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.CROP.TYPE = "absolute"
cfg.INPUT.CROP.SIZE = (720.0, 720.0) # was 720, 1280
cfg.INPUT.RESIZE = False
cfg.INPUT.RESIZE_SHAPE = (720, 2048)
cfg.INPUT.VER_FLIP = True
cfg.INPUT.HOR_FLIP = True
cfg.INPUT.AUG_ON_TEST = False
cfg.INPUT.CONTRAST = True
cfg.INPUT.CONTRAST_RANGE = (.5, 1.5)
cfg.INPUT.BRIGHTNESS = True
cfg.INPUT.BRIGHTNESS_RANGE = (.3, 1.7)
cfg.INPUT.SATURATION = True
cfg.INPUT.SATURATION_RANGE = (.7, 1.4)


output_name = '{}_maxiter-{}_lr-{}_detectPerIm-{}_minsize-{}_batchsize-{}'.format(
    name, cfg.SOLVER.MAX_ITER, cfg.SOLVER.BASE_LR, cfg.TEST.DETECTIONS_PER_IMAGE, 
    np.min(cfg.INPUT.MIN_SIZE_TRAIN), cfg.SOLVER.IMS_PER_BATCH
)
cfg.OUTPUT_DIR = os.path.join(cfg.OUTPUT_DIR, output_name)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Save the config file for future reference
yaml_file = os.path.join(cfg.OUTPUT_DIR, "config.yaml")
with open(yaml_file, 'w') as file:
    file.write(cfg.dump())

./output/LRscheduler-cropped-color-aug-test-new-dataloader_maxiter-80_lr-0.0019_detectPerIm-200_minsize-0_batchsize-8/config.yaml


In [8]:
cfg.OUTPUT_DIR

'./output/LRscheduler-cropped-color-aug-test-new-dataloader_maxiter-80_lr-0.0019_detectPerIm-200_minsize-0_batchsize-8'

In [1]:
# Visualize some of the training images with augmentation and annotations
import matplotlib.pyplot as plt

show_images = True
show_test_images = False
if show_images:
    
    mapper = DetectionDatasetMapper(cfg)
    if show_test_images:
        loader = build_detection_test_loader(cfg, cfg.DATASETS.TEST[0])
    else:
        loader = build_detection_train_loader(cfg, mapper=mapper)
    loader_iter = iter(loader)
    for _ in range(1):
        batch = next(loader_iter)
        for im_dict in batch:
            image = im_dict['image'].numpy()
            image = np.transpose(image, [1, 2, 0])
            plt.figure(figsize=(20,20))
            visualizer = Visualizer(image[:, :, ::-1], 
                                    metadata=train_metadata, 
                                    scale=1.0)

            vis = visualizer.overlay_instances(
                boxes=im_dict['instances'].get('gt_boxes'), 
                labels=im_dict['instances'].get('gt_classes'),
                alpha=1.0)

            plt.imshow(vis.get_image())

In [10]:
trainer = DetectionTrainer(cfg)

[01/10 11:28:55 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [2]:
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Check training quality
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "best-model-config-iter-896-loss-0.8034359399295811.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("val", )
predictor = DefaultPredictor(cfg)

for d in random.sample(val_dicts, 3):
# for d in train_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=train_metadata,
                   scale=0.8,
                   
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(20,20))
    plt.imshow(v.get_image()[:, :, ::])

In [11]:
cfg.OUTPUT_DIR

'./output/LRscheduler-cropped-color-aug_maxiter-6400_lr-0.0019_detectPerIm-200_minsize-0_batchsize-8'